# 新手入门第四课——PaddlePaddle快速入门

CSY 2020-2-12  
https://aistudio.baidu.com/aistudio/projectdetail/267588

## 计算常量的加法：1+1

首先导入PaddlePaddle库

In [1]:
import paddle.fluid as fluid

定义两个张量的常量x1和x2，并指定它们的形状是[2, 2]，并赋值为1铺满整个张量，类型为int64.

In [2]:
# 定义两个张量,仅定义，并没有赋值
x1 = fluid.layers.fill_constant(shape=[2, 2], value=1, dtype='int64') # x1=[[1,1],[1,1]]
x2 = fluid.layers.fill_constant(shape=[2, 2], value=1, dtype='int64') # x2=[[1,1],[1,1]]

In [8]:
print(x1) # 此时x1并没有被赋值

name: "fill_constant_0.tmp_0"
type {
  type: LOD_TENSOR
  lod_tensor {
    tensor {
      data_type: INT64
      dims: 2
      dims: 2
    }
  }
}
persistable: false



定义一个操作，该计算是将上面两个张量进行加法计算，并返回一个求和的算子。

In [10]:
# 定义两个张量的求和操作
y1 = fluid.layers.sum(x=[x1, x2]) # y1=x1+x2

In [11]:
y1 # 只是定义了，并没有运算

name: "sum_1.tmp_0"
type {
  type: LOD_TENSOR
  lod_tensor {
    tensor {
      data_type: INT64
      dims: 2
      dims: 2
    }
    lod_level: 0
  }
}
persistable: false

然后创建一个解释器，可以在这里指定计算使用CPU或GPU。  
当使用CPUPlace()时使用的是CPU  
如果是CUDAPlace()使用的是GPU。

In [12]:
# 创建一个使用CPU的解释器
place = fluid.CPUPlace()
exe = fluid.executor.Executor(place)
# 进行参数初始化
exe.run(fluid.default_startup_program()) # 本例中没有需要初始化的网络参数，故有下面的 warnings

/home/csy/anaconda3/lib/python3.7/site-packages/paddle/fluid/executor.py:790: UserWarning: The current program is empty.
  warnings.warn(error_info)


[]

最后执行计算

In [13]:
# 进行运算，并把y的结果输出
result = exe.run(program=fluid.default_main_program(),
                 fetch_list=[x1,x2,y1])
print('x1',result[0]) # x1
print('x2',result[1]) # x2
print('y1',result[2]) # y1

x1 [[1 1]
 [1 1]]
x2 [[1 1]
 [1 1]]
y1 [[2 2]
 [2 2]]


## 计算变量的加法：1+1

In [14]:
import paddle.fluid as fluid
import numpy as np

定义两个张量，并不指定该张量的形状和值，它们是之后动态赋值的。这里只是指定它们的类型和名字，这个名字是我们之后赋值的关键。

In [18]:
# 定义两个张量
x1 = fluid.layers.create_tensor(dtype='int64', name='a') # 为了省心，大部分程序都将x1用a代替。此处故意区分，可看出不同
x2 = fluid.layers.create_tensor(dtype='int64', name='b') # 为了省心，大部分程序都将x2用b代替。此处故意区分，可看出不同

In [19]:
# 将两个张量求和
y = fluid.layers.sum(x=[x1, x2])

In [20]:
# 创建一个使用CPU的解释器
place = fluid.CPUPlace()
exe = fluid.executor.Executor(place)
# 进行参数初始化
exe.run(fluid.default_startup_program())

/home/csy/anaconda3/lib/python3.7/site-packages/paddle/fluid/executor.py:790: UserWarning: The current program is empty.
  warnings.warn(error_info)


[]

In [21]:
# 定义两个要计算的变量，这是真赋值了
a1 = np.array([3, 2]).astype('int64')
b1 = np.array([1, 1]).astype('int64')

In [25]:
a1.shape

(2,)

In [22]:
a1

array([3, 2])

In [23]:
b1

array([1, 1])

In [24]:
# 进行运算，并把y的结果输出
out_a, out_b, result = exe.run(program=fluid.default_main_program(),
                               feed={'a': a1, 'b': b1},
                               fetch_list=[a, b, y])
print(out_a, " + ", out_b," = ", result)

[3 2]  +  [1 1]  =  [4 3]


## 使用PaddlePaddle做线性回归

In [1]:
import paddle.fluid as fluid
# import paddle
import numpy as np

In [2]:
# 定义一个简单的线性网络
#x = fluid.layers.data(name='x', shape=[13], dtype='float32') # 原文如此
x = fluid.data(name='x', shape=[-1,13], dtype='float32') # 定义输入层的形状为13
hidden = fluid.layers.fc(input=x, size=100, act='relu') # 大小为100，激活函数是ReLU的全连接层
net = fluid.layers.fc(input=hidden, size=1, act=None) # 输出大小为1的全连接层

In [3]:
# 定义损失函数
#y = fluid.layers.data(name='y', shape=[-1,1], dtype='float32')
y = fluid.data(name='y', shape=[-1,1], dtype='float32')
cost = fluid.layers.square_error_cost(input=net, label=y)
avg_cost = fluid.layers.mean(cost)

In [4]:
# 复制一个主程序，方便之后使用
test_program = fluid.default_main_program().clone(for_test=True)

In [5]:
# 定义优化方法
optimizer = fluid.optimizer.SGDOptimizer(learning_rate=0.01)
opts = optimizer.minimize(avg_cost)

In [6]:
# 创建一个使用CPU的解释器
place = fluid.CPUPlace()
exe = fluid.Executor(place)
# 进行参数初始化
exe.run(fluid.default_startup_program())

[]

In [7]:
# 定义训练和测试数据
x_data = np.array([[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 
                   [2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 
                   [3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 
                   [4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 
                   [5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]]).astype('float32')
y_data = np.array([[3.0], [5.0], [7.0], [9.0], [11.0]]).astype('float32')
test_data = np.array([[6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]]).astype('float32')

In [8]:
x_data.shape

(5, 13)

In [9]:
y_data.shape

(5, 1)

In [10]:
test_data.shape

(1, 13)

In [13]:
# 开始训练100个pass
for pass_id in range(50):
    train_cost = exe.run(program=fluid.default_main_program(),
                         feed={'x': x_data, 'y': y_data},
                         fetch_list=[avg_cost])
    print("Pass:%d, Cost:%0.5f" % (pass_id, train_cost[0]))

Pass:0, Cost:0.00220
Pass:1, Cost:0.00216
Pass:2, Cost:0.00211
Pass:3, Cost:0.00207
Pass:4, Cost:0.00203
Pass:5, Cost:0.00199
Pass:6, Cost:0.00195
Pass:7, Cost:0.00191
Pass:8, Cost:0.00187
Pass:9, Cost:0.00183
Pass:10, Cost:0.00179
Pass:11, Cost:0.00176
Pass:12, Cost:0.00172
Pass:13, Cost:0.00169
Pass:14, Cost:0.00165
Pass:15, Cost:0.00162
Pass:16, Cost:0.00159
Pass:17, Cost:0.00156
Pass:18, Cost:0.00152
Pass:19, Cost:0.00149
Pass:20, Cost:0.00146
Pass:21, Cost:0.00143
Pass:22, Cost:0.00141
Pass:23, Cost:0.00138
Pass:24, Cost:0.00135
Pass:25, Cost:0.00132
Pass:26, Cost:0.00130
Pass:27, Cost:0.00127
Pass:28, Cost:0.00124
Pass:29, Cost:0.00122
Pass:30, Cost:0.00119
Pass:31, Cost:0.00117
Pass:32, Cost:0.00115
Pass:33, Cost:0.00112
Pass:34, Cost:0.00110
Pass:35, Cost:0.00108
Pass:36, Cost:0.00106
Pass:37, Cost:0.00104
Pass:38, Cost:0.00101
Pass:39, Cost:0.00099
Pass:40, Cost:0.00097
Pass:41, Cost:0.00095
Pass:42, Cost:0.00094
Pass:43, Cost:0.00092
Pass:44, Cost:0.00090
Pass:45, Cost:0.0008

In [14]:
# 开始预测，‘y’值无所谓，但是必须要有。
result = exe.run(program=test_program,
                 feed={'x': test_data, 'y': np.array([[0.0]]).astype('float32')},
                 fetch_list=[net])
print("当x为6.0时，y为：%0.5f" % result[0][0][0])

当x为6.0时，y为：13.04391
